In [1]:
pip install snscrape

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pymongo

In [3]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import pymongo
import datetime
import json
from datetime import timedelta

# import dataset statistiche

In [4]:
connessione = pymongo.MongoClient("mongodb://localhost:27017/")
WorldCupDb = connessione["WorldCup"] #crea DB
WorldCupCollection = WorldCupDb["WorldCupStats"] #crea collezione all'interno del db

selection=WorldCupCollection.find()
wcMatch=[]
j=0
for i in selection:
    wcMatch.append(i)
    
wcMatch=pd.DataFrame(wcMatch)
#dataset = dataset.rename({'nome_partita': 'partita'}, axis=1)
wcMatch

,_id,tipo_di_partita,squadra_in_casa,squadra_in_trasferta,datetime_kick_off,gol_segnati_casa,gol_segnati_trasferta,rigori_finali_segnati_casa,rigori_finali_segnati_trasferta,nome_partita,...,gol_casa_1,gol_casa_2,gol_casa_3,gol_casa_4,gol_casa_5,gol_casa_6,gol_casa_7,cartellino_rosso_casa_1,gol_trasferta_3,gol_trasferta_4
0,63b835e872176b118ed982e8,Group A,QATAR,ECUADOR,2022-11-20 17:00:00,0,2,0,0,QATAR-ECUADOR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63b835e872176b118ed982e9,Group B,ENGLAND,IRAN,2022-11-21 14:00:00,6,2,0,0,ENGLAND-IRAN,...,Jude BELLINGHAM,Bukayo SAKA,Raheem STERLING,Bukayo SAKA,Marcus RASHFORD,Jack GREALISH,NaN,NaN,NaN,NaN
2,63b835e872176b118ed982ea,Group A,SENEGAL,NETHERLANDS,2022-11-21 17:00:00,0,2,0,0,SENEGAL-NETHERLANDS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63b835e872176b118ed982eb,Group B,UNITED STATES,WALES,2022-11-21 20:00:00,1,1,0,0,UNITED STATES-WALES,...,Tim WEAH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63b835e972176b118ed982ec,Group C,ARGENTINA,SAUDI ARABIA,2022-11-22 11:00:00,1,2,0,0,ARGENTINA-SAUDI ARABIA,...,Lionel MESSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,63b835fd72176b118ed98323,Quarter-final,ENGLAND,FRANCE,2022-12-10 20:00:00,1,2,0,0,ENGLAND-FRANCE,...,Harry KANE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,63b835fe72176b118ed98324,Semi-final,ARGENTINA,CROATIA,2022-12-13 20:00:00,3,0,0,0,ARGENTINA-CROATIA,...,Lionel MESSI,Julian ALVAREZ,Julian ALVAREZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,63b835fe72176b118ed98325,Semi-final,FRANCE,MOROCCO,2022-12-14 20:00:00,2,0,0,0,FRANCE-MOROCCO,...,Theo HERNANDEZ,Randal KOLO MUANI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,63b835ff72176b118ed98326,Play-off for third place,CROATIA,MOROCCO,2022-12-17 16:00:00,2,1,0,0,CROATIA-MOROCCO,...,Josko GVARDIOL,Mislav ORSIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
maxTweets = 999

tweets_list0 = []

#since="2022-11-19"
#until="2022-12-20"
j=-1
for partita in wcMatch['nome_partita']:
    j=j+1
    print(partita)
    since=datetime.datetime.strptime(wcMatch["kick_off_day"][j], "%Y-%m-%d") + datetime.timedelta(days=-1)
    since=since.strftime('%Y-%m-%d')
    until=datetime.datetime.strptime(wcMatch["kick_off_day"][j], "%Y-%m-%d") + datetime.timedelta(days=1)
    until=until.strftime('%Y-%m-%d')
    #since=datetime.datetime.strptime(wcMatch["kick_off_day"][j], "%Y-%m-%d")
    #until=datetime.datetime.strptime(wcMatch["kick_off_day"][j], "%Y-%m-%d")
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper( '{} lang:en since:{} until:{}'.format(partita,since,until)).get_items()):
        if i>maxTweets:
            break
        tweets_list0.append({
          "data_partita":wcMatch["kick_off_day"][j],
         # "id":tweet.id,
         # "username":tweet.user.username,
         # "display_name":tweet.user.displayname,
          "testo":tweet.content, 
          "reply_count":tweet.replyCount,
          "like_count":tweet.likeCount,
          "retweet_count":tweet.retweetCount,
          "quote_count":tweet.quoteCount,
          "hashtags":tweet.hashtags,#puo servire?
          "partita": partita, #Ho pensato di inserirlo per facilitare il raggruppamento in fase di analisi dei sentimenti

      })
df = pd.DataFrame(tweets_list0)
print(df)

QATAR-ECUADOR


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=QATAR-ECUADOR+lang%3Aen+since%3A2022-11-19+until%3A2022-11-21&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel: non-200 status code
4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&i

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=QATAR-ECUADOR+lang%3Aen+since%3A2022-11-19+until%3A2022-11-21&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel failed, giving up.

In [15]:
tweets_list0[11]

{'data_partita': '2022-11-20',
 'testo': 'So this is it! Welcome to FIFA World Cup 2022. Official kick-off started tonight. \nThis is just a quick snap for the opening of FIFA World Cup here in Doha, Qatar. Ecuador won against Qatar for tonight’s match. 2-0 amigos. 😂 #FIFAWorldCup #FIFAWorldCup2022 #QatarWorldCup2022 https://t.co/xfBxSZRCsI',
 'reply_count': 0,
 'like_count': 1,
 'retweet_count': 0,
 'quote_count': 0,
 'hashtags': ['FIFAWorldCup', 'FIFAWorldCup2022', 'QatarWorldCup2022'],
 'partita': 'QATAR-ECUADOR'}

In [18]:
connessione = pymongo.MongoClient("mongodb://localhost:27017/")#stabilisce una connessione
WorldCupDb = connessione["WorldCup"] #crea DB
WorldCupCollection = WorldCupDb["WorldCupTweet"] #crea collezione all'interno del db
for i in range(0,len(tweets_list0)):
    inserimento = WorldCupCollection.insert_one(tweets_list0[i]) # inserisce documento all'interno di collezione

In [13]:
print(connessione.list_database_names())

['WorldCup', 'admin', 'config', 'local']


In [17]:
#WorldCupCollection.delete_many({})